## Voraussetzungen

1. Internetverbindung
2. Python 3 Version (getestet mit 3.11.3)
3. Installierte Module

## Beschreibung

Dieses Notebook dient der Aggregierung der Faktoren, auf dessen Basis der probabilistische Algorithmus bestimmen soll, ob ein Tile in den Cache geladen werden soll oder nicht. Folgende Aktionen werden durchgeführt:

1. (Stündliche) Wetterdaten des Jahres 2023 werden von [meteostat.net](https://meteostat.net) heruntergeladen.
2. Feiertage und Schulferien werden aus den Kalenderdaten des Jahres 2023 extrahiert. Diese wurden von [feiertage-deutschland.de](https://www.feiertage-deutschland.de) installiert.
3. Nicht für die Klassifizierung benötigte Wetterinformationen werden entfernt.
4. Stetige Daten werden in diskrete Daten umgewandelt (Binning).
5. Gesammelte Daten werden zusammengefasst.
6. Daten werden gebündelt in die Datei `./data/agg_mapped_data.csv` exportiert.

In [1]:
# Importieren von Python Modulen

from meteostat import Point, Hourly
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import ics

#### 1. Wetterdaten empfangen

Wir nutzen das `meteostat`-Modul, welches von [meteostat.net](https://dev.meteostat.net/) angeboten wird. Wir empfangen stündliche Wetterinformationen für Berlin über den Zeitraum vom 01.01.2023 - 31.12.2023 (inklusive).

In [2]:
# Zeitraum festlegen
start = datetime(2023, 1, 1)
end = datetime(2024, 1, 1) - timedelta(hours=1)

# Wetterstation festlegen
berlin = Point(52.5068042, 13.0950944, 10)

# Daten abfragen
weather_data = Hourly(berlin, start, end).fetch()

#### 2. Feiertage und Schulferien

Die Ferien und Feiertage wurden von [feiertage-deutschland.de](https://www.feiertage-deutschland.de/kalender-download/) als Kalender-Datei (`.ics`) heruntergeladen. Mithilfe des `ics`-Moduls werden diese ausgelesen und verwertet.

In [3]:
# Gibt für den angegebenen Zeitrahmen alle Stunden zurück, in denen ein Ereignis vermerkt ist
def parse_ics(path: str, from_date, to_date) -> pd.DataFrame:
    # Stündliche Daten, daher werden die Tage mit 24 multipliziert  
    has_event = np.zeros((to_date-from_date).days * 24)
    valid_events = [] # Events die in dem Zeitbereich liegen
    
    with open(path, 'r') as fp:
        calendar = ics.Calendar(fp.read())
        for event in calendar.events:
            start_time = event.begin.datetime.replace(tzinfo=None)
            end_time = event.end.datetime.replace(tzinfo=None)
            if (from_date <= start_time <= to_date) or (from_date <= end_time <= to_date):
                valid_events.append(event)

    for i in range(len(has_event)):
        current_time = (from_date + timedelta(hours=i))

        for event in valid_events:
            begin = event.begin.datetime.replace(tzinfo=None)
            end = event.end.datetime.replace(tzinfo=None)
            if begin <= current_time <= end:
                has_event[i] = 1

    return has_event

In [4]:
is_vacation = parse_ics('data/schulferien-berlin.ics', start, datetime(2024, 1, 1))
is_holiday = parse_ics('data/feiertage-deutschland.ics', start, datetime(2024, 1, 1))

#### 3. Ungewünschte Wetterdaten entfernen

Die Beschreibung der Felder in der Wettertabelle kann [dev.meteostat.net/bulk/hourly.html#structure](https://dev.meteostat.net/bulk/hourly.html#structure) entnommen werden

In [5]:
weather_data_cleaned = weather_data.copy()
weather_data_cleaned.drop(columns=['dwpt', 'rhum', 'wdir', 'wpgt', 'pres', 'tsun', 'prcp'], inplace=True)

#### 4. Binning der Daten

Das Binning wird genutzt, um stetige Daten in eine diskrete Form zu bringen. Dazu werden Intervalle beschlossen, in welchen den stetigen Daten jeweils die gleiche Kategorie (Bin) zugeordnet wird.

Folgende Bins werden erstellt:

- Temperatur (`temp`) in $°\text{C}$  wird aufgeteilt in:
  - Frieren(0): $(-\infty, 0]$
  - Kühl(1): $(0, 10]$,
  - Moderat(2): $(10, 20]$
  - Warm(3): $(20, 28]$
  - Heiß(4): $(28, +\infty)$

In [6]:
def map_temp(temp: float) -> int:
    if temp <= 0:
        return 0
    elif temp <= 10:
        return 1
    elif temp <= 20:
        return 2
    elif temp <= 28:
        return 3
    elif temp > 28:
        return 4
    else:
        raise ValueError("Received unexpected value: {}".format(temp))

- Schneehöhe (`snow`) in $\text{cm}$
    - Kein Schnee(0): $[0]$
    - Leicht(1): $(0, 10]$
    - Mäßig(2): $(10, 30]$
    - Hoch(3): $(30, 50]$
    - Sehr Hoch(4): $(50, +\infty)$

In [7]:
def map_snow(snow: float) -> int:
    if snow == 0:
        return 0
    elif snow <= 10:
        return 1
    elif snow <= 30:
        return 2
    elif snow <= 50:
        return 3
    elif snow > 50:
        return 4
    else:
        raise ValueError("Received unexpected value: {}".format(snow))

- Windgeschwindigkeit (`wspd`) in $\text{km/h}$
    - Windstill(0): $[0, 5]$
    - Leichte Brise(1): $(5, 20]$
    - Mäßiger Wind(2): $(20, 40]$
    - Starker Wind(3): $(40, 60]$
    - Sturm(4): $(60, +\infty)$

In [8]:
def map_wind(wspd: float) -> int:
    if 0 <= wspd <= 5:
        return 0
    elif wspd <= 20:
        return 1
    elif wspd <= 40:
        return 2
    elif wspd <= 60:
        return 3
    elif wspd > 60:
        return 4
    else:
        raise ValueError("Received unexpected value: {}".format(wspd))

- Wetterverhältnisse (`coco`):
    - Liste der Codes kann unter https://dev.meteostat.net/formats.html#weather-condition-codes eingesehen werden.
      Diese werden zur Einfachheit in die Kategorien Schlecht(0), Neutral(1), Gut(2) eingeteilt

In [9]:
def map_coco(coco: float) -> int:
    if coco in [3.0, 4.0, 5.0, 7.0, 14.0, 17.0, 21.0]:
        return 1
    elif coco in [1.0, 2.0]:
        return 2
    elif coco in [6.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 15.0, 16.0, 18.0, 19.0, 20.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0]:
        return 0
    else:
        raise ValueError("Received unexpected value: {}".format(coco))

Die erstellten Funktionen werden nun auf die Wetterdaten mittels angewendet.

In [10]:
weather_data_cleaned['coco'] = weather_data_cleaned['coco'].map(map_coco)
weather_data_cleaned['temp'] = weather_data_cleaned['temp'].map(map_temp)
weather_data_cleaned['snow'] = weather_data_cleaned['snow'].map(map_snow)
weather_data_cleaned['wspd'] = weather_data_cleaned['wspd'].map(map_wind)

Zeit wird in die Spalten Stunde (`hour`), Wochentag (`weekday`) und Monat (`month`) aufgeteilt.

In [11]:
weather_data_cleaned.reset_index(inplace=True)
weather_data_cleaned['weekday'] = weather_data_cleaned['time'].map(lambda t: t.weekday())
weather_data_cleaned['month'] = weather_data_cleaned['time'].map(lambda t: t.month)
weather_data_cleaned['hour'] = weather_data_cleaned['time'].map(lambda t: t.hour)
weather_data_cleaned.drop(columns=['time'], inplace=True)
weather_data_cleaned.index.name = 'index'

#### 5. Gesammelte Daten werden zusammengefasst

Die Ferien- und Feiertagsdaten werden der Tabelle zugeführt

In [12]:
weather_data_cleaned['vacation'] = is_vacation.astype(int)
weather_data_cleaned['holiday'] = is_holiday.astype(int)

#### 6. Datenexport

Die Daten werden für die weitere Verwendung in die Datei `./data/agg_mapped_data.csv` exportiert.

In [13]:
weather_data_cleaned.to_csv('./data/agg_mapped_data.csv')